In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Models to do transfer learning on
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# Load train and testdata

In [ ]:
# Load and preprocess fashion mnist data for benchmark model
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

# Benchmark model
Don't run this code (it takes a long time to run).

Perfpormance: 52.6% on testdata.

In [7]:
# Reshape and normalize the images
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

In [12]:
def build_benchmark_model():
    model = Sequential([
        Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
    
        Conv2D(128, (3, 3), padding='same', activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.4),
    
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.5),
    
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.6),
        Dense(100, activation='softmax')  # 100 classes in CIFAR-100
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [16]:
benchmark_model = build_benchmark_model()

In [17]:
benchmark_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 16, 16, 64)        0         
                                                                 
 conv2d_16 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 8, 8, 128)         0         
 ng2D)                                                           
                                                                 
 dropout_13 (Dropout)        (None, 8, 8, 128)        

In [18]:
benchmark_model.fit(train_images, train_labels, epochs=200, batch_size=256, verbose=1)

Epoch 1/200
196/196 [==============================] - 64s 322ms/step - loss: 4.5876 - accuracy: 0.0158
Epoch 2/200
196/196 [==============================] - 65s 332ms/step - loss: 4.3172 - accuracy: 0.0417
Epoch 3/200
196/196 [==============================] - 65s 333ms/step - loss: 4.1091 - accuracy: 0.0710
Epoch 4/200
196/196 [==============================] - 66s 337ms/step - loss: 3.9345 - accuracy: 0.1005
Epoch 5/200
196/196 [==============================] - 65s 334ms/step - loss: 3.8032 - accuracy: 0.1190
Epoch 6/200
196/196 [==============================] - 66s 336ms/step - loss: 3.6865 - accuracy: 0.1387
Epoch 7/200
196/196 [==============================] - 66s 334ms/step - loss: 3.5993 - accuracy: 0.1514
Epoch 8/200
196/196 [==============================] - 66s 336ms/step - loss: 3.5200 - accuracy: 0.1664
Epoch 9/200
196/196 [==============================] - 66s 338ms/step - loss: 3.4506 - accuracy: 0.1793
Epoch 10/200
196/196 [==============================] - 66s 337m

In [19]:
test_loss, test_acc = benchmark_model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_acc}")

313/313 - 4s - loss: 1.8781 - accuracy: 0.5256 - 4s/epoch - 12ms/step
Test Accuracy: 0.525600016117096


# Task
Do transfer learning on the different models you can find in the import. If you have other models you want to try, go ahead!

The models should be trained on [CIFAR-100](https://www.cs.toronto.edu/~kriz/cifar.html) dataset.

Use imagenet weights for the models (if it dosen't exist, use something else that is not cifar100).

Do both fine tuning and feature extraction.

- Which models can you get to beat the testscore of the benchmark model?
- Which model can you get the highest accuracy on?


In [166]:
(train_images_x, train_labels_y), (test_images_x, test_labels_y) = cifar100.load_data()
train_images = tf.keras.applications.vgg16.preprocess_input(train_images_x)
test_images = tf.keras.applications.vgg16.preprocess_input(test_images_x)
train_labels = to_categorical(train_labels_y)
test_labels = to_categorical(test_labels_y)

In [167]:
# Load pre-trained VGG16 model without the top layer
def create_vgg_feature_extraction_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    # Freeze the base model
    base_model.trainable = False
    
    # Create a new model on top
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = base_model(inputs)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(100, activation='softmax')(x)
    model = Model(inputs, outputs)
    
    # Compile and train the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [168]:
model = create_vgg_feature_extraction_model()

In [ ]:
model.fit(train_images, train_labels, epochs=100)

Epoch 1/100
1563/1563 [==============================] - 109s 69ms/step - loss: 4.1518 - accuracy: 0.1400
Epoch 2/100
1563/1563 [==============================] - 112s 72ms/step - loss: 2.9788 - accuracy: 0.2615
Epoch 3/100
1563/1563 [==============================] - 119s 76ms/step - loss: 2.7075 - accuracy: 0.3077
Epoch 4/100
1563/1563 [==============================] - 113s 72ms/step - loss: 2.5644 - accuracy: 0.3360
Epoch 5/100
1563/1563 [==============================] - 114s 73ms/step - loss: 2.4711 - accuracy: 0.3538
Epoch 6/100
1563/1563 [==============================] - 114s 73ms/step - loss: 2.3965 - accuracy: 0.3693
Epoch 7/100
1563/1563 [==============================] - 113s 73ms/step - loss: 2.3451 - accuracy: 0.3824
Epoch 8/100
1563/1563 [==============================] - 113s 72ms/step - loss: 2.3043 - accuracy: 0.3895
Epoch 9/100
1563/1563 [==============================] - 113s 73ms/step - loss: 2.2721 - accuracy: 0.3961
Epoch 10/100
1563/1563 [======================